### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [ ]:
# Write your code from here

In [1]:
pip install grpcio grpcio-tools


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import time
import grpc
from concurrent import futures

# Dynamically create the .proto file
proto_code = """
syntax = "proto3";
package validation;

service Validator {
  rpc Validate (ValidationRequest) returns (ValidationResponse);
}

message ValidationRequest {
  string name = 1;
  int32 age = 2;
}

message ValidationResponse {
  bool is_valid = 1;
  string message = 2;
}
"""

# Save .proto to disk
proto_filename = "validation.proto"
with open(proto_filename, "w") as f:
    f.write(proto_code)

# Generate Python gRPC code
from grpc_tools import protoc
protoc.main((
    '',
    f'-I.',
    f'--python_out=.',
    f'--grpc_python_out=.',
    proto_filename,
))

# Import generated modules
import validation_pb2
import validation_pb2_grpc


# Server implementation
class ValidatorServicer(validation_pb2_grpc.ValidatorServicer):
    def Validate(self, request, context):
        print(f"Received: name={request.name}, age={request.age}")
        if not request.name or request.age < 0:
            return validation_pb2.ValidationResponse(
                is_valid=False,
                message="❌ Invalid input: name required, age must be >= 0"
            )
        return validation_pb2.ValidationResponse(
            is_valid=True,
            message="✅ Validation successful"
        )


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validation_pb2_grpc.add_ValidatorServicer_to_server(ValidatorServicer(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    print("🟢 gRPC server started on port 50051")
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        print("🛑 Server stopped")
        server.stop(0)


if __name__ == "__main__":
    serve()


🟢 gRPC server started on port 50051
🛑 Server stopped
